In [1]:

import sqlite3

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [2]:
DBPATH = 'pradb.db'
conn = sqlite3.connect(DBPATH)

In [3]:
cursor = conn.cursor()

omb_control_number,icr_reference_number,Title,AgencyCode,ICRTypeCode,expiration_date,ICR_status,file_date

In [4]:
sql = (
    "CREATE TABLE IF NOT EXISTS inventory ("
    "    omb_control_number TEXT NOT NULL,"
    "    icr_reference_number TEXT NOT NULL,"
    "    Title TEXT NOT NULL,"
    "    AgencyCode TEXT NOT NULL,"
    "    ICRTypeCode TEXT NOT NULL,"
    "    expiration_date TEXT NOT NULL,"
    "    ICR_status TEXT NOT NULL,"
    "    file_date TEXT NOT NULL"
    ")"
)

cursor.execute(sql)

In [6]:
# find the information, save as csv for rmarkdown to read later
import pycurl
import requests
import os 
from lxml import etree
import pandas as pd 
import datetime
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import shutil


dol_agency_codes = [
1220,
1225,
1205,
1219,
1218,
1250,
1245,
1240,
1293,
1290,
1235,
1210]

def get_xpath_strings(dol_agency_codes):       
    xpath_strings = []
    xpath_string_prefix = '//InformationCollectionRequest[.//AgencyCode/text()[. = "'
    xpath_string_suffix = '"]]'
    
    for number in dol_agency_codes:
        new_string = xpath_string_prefix+str(number)+xpath_string_suffix
        xpath_strings.append(new_string) 
    
    return(xpath_strings)
    

def inventory_to_list(file,dol_agency_codes):
    xpath_strings= get_xpath_strings(dol_agency_codes)
    
    tree = etree.parse(file)
    root = tree.getroot()
    all_results = []

    for string in xpath_strings:
      results = []
      req = root.xpath(string)
      for request in req:       
        try:
         results.append({
           "omb_control_number": request.xpath('./OMBControlNumber//text()')[0],
           "icr_reference_number": request.xpath('./ICRReferenceNumber//text()')[0],
           "Title": request.xpath('./Title//text()')[0],
           #"Abstract": request.xpath('./Abstract//text()')[0],
           "AgencyCode": str(request.xpath('./AgencyCode//text()')[0]).strip(),
           "ICRTypeCode": str(request.xpath('./ICRTypeCode//text()')[0]).strip(),
           "expiration_date": str(request.xpath('./Expiration/ExpirationDate//text()')[0]).strip(),
           "ICR_status": str(request.xpath('./ICRStatus//text()')[0]).strip()})
        except:
            results.append({
              "omb_control_number": request.xpath('./OMBControlNumber//text()')[0],
              "icr_reference_number": request.xpath('./ICRReferenceNumber//text()')[0],
              "Title": request.xpath('./Title//text()')[0],
              #"Abstract": request.xpath('./Abstract//text()')[0],
              "AgencyCode": str(request.xpath('./AgencyCode//text()')[0]).strip(),
              "ICRTypeCode": str(request.xpath('./ICRTypeCode//text()')[0]).strip(),
              "expiration_date": str(request.xpath('./Expiration/ExpirationDate//text()')[0]).strip(),
              "ICR_status": "None"})
            
      all_results.append(results) 
    return(all_results)
    
    
    
def inventory_list_to_table(file,all_results):    
    dol_inventory_row_list = []
    
    for list in all_results:
      for row in list:
        dol_inventory_row_list.append(row)
    
    dol_inventory_table = pd.DataFrame(dol_inventory_row_list)
    dol_inventory_table['file_date'] = file
    return(dol_inventory_table)   

In [7]:
os.getcwd()

'/home/jupyter-ed/projects/pra/data/01_database'

In [8]:
os.chdir('../inventory')

In [9]:
os.listdir()

['2020-07-01_inventory.xml',
 'new_in_pending_today_table.csv',
 '2020-07-06_inventory.xml',
 '2020-07-10_inventory.xml',
 'pending.csv',
 '.ipynb_checkpoints',
 '2020-06-23_inventory.xml',
 '2020-07-02_inventory.xml',
 '2020-06-25_inventory.xml',
 '2020-06-28_inventory.xml',
 '2020-07-18_inventory.xml',
 '2020-07-16_inventory.xml',
 '2020-07-26_inventory.xml',
 '2020-07-14_inventory.xml',
 '2020-07-15_inventory.xml',
 '2020-07-11_inventory.xml',
 '2020-07-24_inventory.xml',
 '2020-07-17_inventory.xml',
 '2020-07-12_inventory.xml',
 '2020-07-08_inventory.xml',
 '2020-07-25_inventory.xml',
 '2020-07-13_inventory.xml',
 '2020-06-29_inventory.xml',
 '2020-07-20_inventory.xml',
 '2020-07-09_inventory.xml',
 '2020-06-27_inventory.xml',
 '2020-07-21_inventory.xml',
 '2020-07-23_inventory.xml',
 '2020-06-24_inventory.xml',
 '2020-07-05_inventory.xml',
 '2020-07-19_inventory.xml',
 '2020-07-22_inventory.xml',
 '2020-07-04_inventory.xml',
 'gone_from_pending_today_table.csv',
 '2020-07-27_inven

In [11]:
inventory=inventory_to_list('2020-06-23_inventory.xml',dol_agency_codes)

In [12]:
inv_df = inventory_list_to_table('2020-06-23_inventory.xml',inventory)

In [13]:
inv_df

,omb_control_number,icr_reference_number,Title,AgencyCode,ICRTypeCode,expiration_date,ICR_status,file_date
0,1220-0008,201705-1220-001,Producer Price Index Survey,1220,Extension without change of a currently approv...,2020-10-31-04:00,Active,2020-06-23_inventory.xml
1,1220-0011,202003-1220-001,Report on Current Employment Statistics,1220,No material or nonsubstantive change to a curr...,2020-12-31-05:00,Active,2020-06-23_inventory.xml
2,1220-0012,201708-1220-001,Quarterly Census of Employment and Wages (QCEW),1220,Extension without change of a currently approv...,2021-02-28-05:00,Active,2020-06-23_inventory.xml
3,1220-0017,201712-1220-002,Local Area Unemployment Statistics Program,1220,Revision of a currently approved collection,2021-07-31-04:00,Active,2020-06-23_inventory.xml
4,1220-0025,201806-1220-002,International Price Program U.S. Import and Ex...,1220,Extension without change of a currently approv...,2022-05-31-04:00,Active,2020-06-23_inventory.xml
...,...,...,...,...,...,...,...,...
467,1210-0150,201803-1210-001,Coverage of Certain Preventive Services under ...,1210,Revision of a currently approved collection,2021-11-30-05:00,Active,2020-06-23_inventory.xml
468,1210-0153,201902-1210-001,Alternative Reporting Methods for Apprenticesh...,1210,Revision of a currently approved collection,2022-08-31-04:00,Active,2020-06-23_inventory.xml
469,1210-0158,201906-1210-004,Insurance and Annuity Contracts and Mutual Fun...,1210,Revision of a currently approved collection,2022-08-31-04:00,Active,2020-06-23_inventory.xml
470,1210-0160,201905-1210-001,Notice for Health Reimbursement Arrangements i...,1210,New collection (Request for a new OMB Control ...,2022-06-30-04:00,Active,2020-06-23_inventory.xml


In [14]:
inv_df.columns

Index(['omb_control_number', 'icr_reference_number', 'Title', 'AgencyCode',
       'ICRTypeCode', 'expiration_date', 'ICR_status', 'file_date'],
      dtype='object')

In [16]:
inv_df.to_sql('inv', con=engine)

In [17]:
engine.execute("SELECT * FROM inv").fetchall()

[(0, '1220-0008', '201705-1220-001', 'Producer Price Index Survey', '1220', 'Extension without change of a currently approved collection', '2020-10-31-04:00', 'Active', '2020-06-23_inventory.xml'),
 (1, '1220-0011', '202003-1220-001', 'Report on Current Employment Statistics', '1220', 'No material or nonsubstantive change to a currently approved collection', '2020-12-31-05:00', 'Active', '2020-06-23_inventory.xml'),
 (2, '1220-0012', '201708-1220-001', 'Quarterly Census of Employment and Wages (QCEW)', '1220', 'Extension without change of a currently approved collection', '2021-02-28-05:00', 'Active', '2020-06-23_inventory.xml'),
 (3, '1220-0017', '201712-1220-002', 'Local Area Unemployment Statistics Program', '1220', 'Revision of a currently approved collection', '2021-07-31-04:00', 'Active', '2020-06-23_inventory.xml'),
 (4, '1220-0025', '201806-1220-002', 'International Price Program U.S. Import and Export Price Indexes', '1220', 'Extension without change of a currently approved co

In [18]:
engine.execute("SELECT count() FROM inv").fetchall()

[(472,)]

In [19]:
inv_df.to_sql('inv', conn, if_exists='replace', index = False)

In [21]:
cursor.execute('''  
SELECT * FROM inv
          ''')

for row in cursor.fetchall():
    print (row)

('1220-0008', '201705-1220-001', 'Producer Price Index Survey', '1220', 'Extension without change of a currently approved collection', '2020-10-31-04:00', 'Active', '2020-06-23_inventory.xml')
('1220-0011', '202003-1220-001', 'Report on Current Employment Statistics', '1220', 'No material or nonsubstantive change to a currently approved collection', '2020-12-31-05:00', 'Active', '2020-06-23_inventory.xml')
('1220-0012', '201708-1220-001', 'Quarterly Census of Employment and Wages (QCEW)', '1220', 'Extension without change of a currently approved collection', '2021-02-28-05:00', 'Active', '2020-06-23_inventory.xml')
('1220-0017', '201712-1220-002', 'Local Area Unemployment Statistics Program', '1220', 'Revision of a currently approved collection', '2021-07-31-04:00', 'Active', '2020-06-23_inventory.xml')
('1220-0025', '201806-1220-002', 'International Price Program U.S. Import and Export Price Indexes', '1220', 'Extension without change of a currently approved collection', '2022-05-31-0